# 用于预训练BERT的数据集
:label:`sec_bert-dataset`

为了预训练如:numref:`sec_bert`中实现的BERT模型，我们需要生成理想格式的数据集以促进两个预训练任务：掩码语言建模和下一句预测。一方面，原始的BERT模型是在BookCorpus和English Wikipedia这两个巨大语料库的合并上进行预训练的（见:numref:`subsec_bert_pretraining_tasks`），这使得大多数本书的读者难以运行。另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上预训练BERT变得越来越流行。为了便于演示BERT预训练，我们使用了一个较小的语料库WikiText-2 :cite:`Merity.Xiong.Bradbury.ea.2016`。

与用于预训练word2vec的PTB数据集相比（见:numref:`sec_word2vec_data`），WikiText-2 (i)保留了原始标点符号，使其适合下一句预测；(ii)保留了原始大小写和数字；(iii)规模超过两倍大。

In [1]:
import os
import random
import torch
from d2l import torch as d2l

在[**the WikiText-2 dataset**]中，
每一行代表一个段落，其中
任何标点符号与其前面的词之间都插入了空格。
保留至少包含两个句子的段落。
为了简化，我们仅使用句号作为分隔符来分割句子。
更复杂的句子分割技术讨论将在本节末尾的练习中进行。

In [2]:
#@save
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

#@save
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # Uppercase letters are converted to lowercase ones
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

## 定义预训练任务的辅助函数

在接下来的内容中，
我们首先实现两个BERT预训练任务的辅助函数：
下一句预测和掩码语言模型。
当将原始文本语料转换为适合预训练BERT的理想格式的数据集时，
这些辅助函数将会被调用。

### [**生成下一句预测任务**]

根据:numref:`subsec_nsp`中的描述，
`_get_next_sentence`函数为二分类任务生成一个训练样本。

In [3]:
#@save
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # `paragraphs` is a list of lists of lists
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

以下函数通过调用 `_get_next_sentence` 函数从输入 `paragraph` 中生成用于下一句预测的训练样本。这里 `paragraph` 是一个句子列表，每个句子都是一个词元列表。参数 `max_len` 指定了预训练期间 BERT 输入序列的最大长度。

In [4]:
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # Consider 1 '<cls>' token and 2 '<sep>' tokens
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

### [**生成掩码语言模型任务**]
:label:`subsec_prepare_mlm_data`

为了从BERT输入序列生成用于掩码语言模型任务的训练样本，我们定义了以下 `_replace_mlm_tokens` 函数。在其输入中，`tokens` 是表示BERT输入序列的标记列表，`candidate_pred_positions` 是BERT输入序列的标记索引列表，不包括特殊标记（在掩码语言模型任务中不会预测特殊标记），`num_mlm_preds` 表示预测的数量（回想15%的随机标记进行预测）。根据:numref:`subsec_mlm` 中对掩码语言模型任务的定义，在每个预测位置，输入可以被替换为特殊的“&lt;mask&gt;”标记或随机标记，或者保持不变。最终，该函数返回可能替换后的输入标记、发生预测的标记索引以及这些预测的标签。

In [5]:
#@save
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # For the input of a masked language model, make a new copy of tokens and
    # replace some of them by '<mask>' or random tokens
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # Shuffle for getting 15% random tokens for prediction in the masked
    # language modeling task
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80% of the time: replace the word with the '<mask>' token
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10% of the time: keep the word unchanged
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10% of the time: replace the word with a random word
            else:
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

通过调用上述 `_replace_mlm_tokens` 函数，以下函数将 BERT 输入序列 (`tokens`) 作为输入，并返回输入 tokens 的索引（如 :numref:`subsec_mlm` 中所述，可能经过替换后），进行预测的 token 索引，以及这些预测的标签索引。

In [6]:
#@save
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # `tokens` is a list of strings
    for i, token in enumerate(tokens):
        # Special tokens are not predicted in the masked language modeling
        # task
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 15% of random tokens are predicted in the masked language modeling task
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

## 将文本转换为预训练数据集

现在我们几乎已经准备好为预训练 BERT 定制一个 `Dataset` 类。
在此之前，
我们还需要定义一个辅助函数 `_pad_bert_inputs`
来[**在输入中添加特殊的“&lt;pad&gt;”标记。**]
它的参数 `examples` 包含了来自两个预训练任务的辅助函数 `_get_nsp_data_from_paragraph` 和 `_get_mlm_data_from_tokens` 的输出。

In [7]:
#@save
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # `valid_lens` excludes count of '<pad>' tokens
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # Predictions of padded tokens will be filtered out in the loss via
        # multiplication of 0 weights
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

将用于生成两个预训练任务的训练样本的辅助函数，以及用于填充输入的辅助函数放在一起，我们定制了以下 `_WikiTextDataset` 类作为[**预训练 BERT 的 WikiText-2 数据集**]。通过实现 `__getitem__` 函数，我们可以任意访问从 WikiText-2 语料库中的一对句子生成的预训练（掩码语言模型和下一句预测）示例。

原始的 BERT 模型使用词汇量为 30000 的 WordPiece 嵌入 :cite:`Wu.Schuster.Chen.ea.2016`。WordPiece 的分词方法是对 :numref:`subsec_Byte_Pair_Encoding` 中的原始字节对编码算法的轻微修改。为了简化，我们使用 `d2l.tokenize` 函数进行分词。出现次数少于五次的不常见标记将被过滤掉。

In [8]:
#@save
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # Input `paragraphs[i]` is a list of sentence strings representing a
        # paragraph; while output `paragraphs[i]` is a list of sentences
        # representing a paragraph, where each sentence is a list of tokens
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # Get data for the next sentence prediction task
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # Get data for the masked language model task
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # Pad inputs
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

通过使用 `_read_wiki` 函数和 `_WikiTextDataset` 类，我们定义了以下 `load_data_wiki` 来[**下载 WikiText-2 数据集并从中生成预训练示例**]。

In [9]:
#@save
def load_data_wiki(batch_size, max_len):
    """Load the WikiText-2 dataset."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

将批量大小设置为512，BERT输入序列的最大长度设置为64，我们[**打印出一批BERT预训练示例的形状**]。请注意，在每个BERT输入序列中，有$10$（$64 \times 0.15$）个位置被预测用于掩码语言建模任务。

In [10]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

BadZipFile: File is not a zip file

最后，让我们来看看词汇量。
即使过滤掉不常见的词，
它仍然比PTB数据集的词汇量大两倍多。

In [11]:
len(vocab)

20256

## 摘要

* 与PTB数据集相比，WikiText-2数据集保留了原始的标点符号、大小写和数字，并且规模超过两倍。
* 我们可以任意访问从WikiText-2语料库中的一对句子生成的预训练（掩码语言建模和下一句预测）示例。

## 练习

1. 为了简化起见，这里仅使用句号作为分隔句子的唯一分隔符。尝试其他句子分割技术，例如spaCy和NLTK。以NLTK为例。首先需要安装NLTK：`pip install nltk`。在代码中，先`import nltk`。然后，下载Punkt句子分词器：`nltk.download('punkt')`。要分割如`sentences = 'This is great ! Why not ?'`这样的句子，调用`nltk.tokenize.sent_tokenize(sentences)`将返回一个包含两个句子字符串的列表：`['This is great !', 'Why not ?']`。
1. 如果我们不过滤掉任何低频词，词汇表的大小是多少？

[讨论](https://discuss.d2l.ai/t/1496)